In [4]:
import pandas as pd
import os
from census import Census


In [5]:
api_key = os.getenv('CENSUS_KEY','no variable found')
print(f'API Key: {api_key}')

county_ids = [53033,53035,53053,53061]
state_id = 53

c = Census(api_key)

API Key: 1b373fe94cdee2c96c20b3c1f14ed6b1d6b92562


In [11]:
dec_cols_dict = {
    'P1_001N':'total_population',
    'H1_001N':'housing_units',
    'H1_002N':'households',
    'P5_001N':'group_quarters'
}

In [16]:
df = (
    pd.DataFrame(c.pl.state_place(['NAME','P1_001N','H1_001N','H1_002N','P5_001N'], state_id, '*'))
    .rename(columns=dec_cols_dict)
    .assign(
        place_id = lambda x: (x['state'].astype(str) + x['place'].astype(str)).astype('int64'),
    )
    .drop(columns=['state','place'])
)

df

,NAME,total_population,housing_units,households,group_quarters,place_id
0,"Aberdeen city, Washington",17013.0,7236.0,6449.0,413.0,5300100
1,"Aberdeen Gardens CDP, Washington",279.0,123.0,120.0,0.0,5300135
2,"Acme CDP, Washington",229.0,100.0,93.0,0.0,5300275
3,"Addy CDP, Washington",210.0,100.0,83.0,0.0,5300380
4,"Ahtanum CDP, Washington",4046.0,1532.0,1468.0,0.0,5300800
...,...,...,...,...,...,...
634,"Yacolt town, Washington",1668.0,541.0,523.0,0.0,5379975
635,"Yakima city, Washington",96968.0,37192.0,35379.0,2078.0,5380010
636,"Yarrow Point town, Washington",1134.0,412.0,387.0,0.0,5380150
637,"Yelm city, Washington",10617.0,3455.0,3356.0,82.0,5380220
